## 2、店铺分析

In [1]:
import pymysql
import pandas as pd

# 连接数据库
db = pymysql.connect(host='localhost',port=3306,user='root',passwd='qwer',db='dataany')
# 获取游标
cursor = db.cursor()
tbsql = """
select * from dataany.taobao 
"""
tmsql = "select * from dataany.tianmao"

tbdf = pd.read_sql_query(tbsql,con=db) #tbdf即为淘宝的dataframe
tmdf = pd.read_sql_query(tmsql,con=db)
cursor.close()
db.close()

In [70]:
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行

### （1）淘宝、天猫店铺数与销售额关系

In [6]:
tmdf.head(2)

,店铺id,店铺名称,位置,商品id,店铺类型,一级类目,二级类目,三级类目,四级类目,叶子类目,...,属性:限期使用日期范围,属性:面部防晒单品,属性:保质期,属性:适用对象,属性:功效,属性:月份,属性:PA值,属性:上市时间,属性:是否为特殊用途化妆品,属性:化妆品净含量
0,420751185,美一佳化妆品专营店,江苏 徐州,567119376040,B,美容护肤/美体/精油,防晒,null,null,防晒,...,"2020-11-20至2021-01-29""",美白防晒乳液促销装,null,所有人群,美白 控油 防晒 隔离 提亮肤色,null,PA+++,2018,是,90g
1,389335512,露得清官方旗舰店,浙江 嘉兴,568286063319,B,美容护肤/美体/精油,防晒,null,null,防晒,...,null,null,null,null,null,null,null,null,null,null


天猫：58345条数据，其中包含了959个品牌（包含空值品牌），1694家店铺，8519个不同的商品id（注：同一商品在不同店铺会有不同商品id），销量40970804单，销售额3404016928.53元。
淘宝：739236条数据，其中包含55374家店铺，198448个商品，2679个品牌，销量28128437单，销售额2203345268.36元

单店月产出：
天猫：133963.7
淘宝：2652.7

In [15]:
3404016928.53/1694/15

133963.672905549

In [16]:
2203345268.36/55374/15

2652.6832910270764

In [39]:
import numpy as np
a= tmdf.pivot_table(index=['店铺类型'],values=['销售额','销量'],aggfunc=np.sum)
a

,销售额,销量
店铺类型,,
B,3.404017e+09,40970804


In [54]:
b = tbdf.pivot_table(index=['店铺类型'],values=['销售额','销量'],aggfunc=np.sum)
b

,销售额,销量
店铺类型,,
C,2.203345e+09,28128437


In [42]:
pd.concat([a,b])

,销售额,销量
店铺类型,,
B,3.404017e+09,40970804
C,2.203345e+09,28128437


### （2）淘宝、天猫topn店铺数占比月度变化

In [4]:
import numpy as np
tmdp = tmdf.pivot_table(index=['店铺id'],columns=['日期'],values=['销售额'],aggfunc=np.sum,margins=1,fill_value=0)

In [13]:
import numpy as np
tbdf.pivot_table(index=['店铺id'],columns=['日期'],values=['销售额'],aggfunc=np.sum,margins=1,fill_value=0)

### （3）店铺地域分布及销量分布

In [13]:
import numpy as np
tmdf1 = tmdf[['店铺id','位置','销售额','日期','商品id']]
tmdf1.head()

,店铺id,位置,销售额,日期,商品id
0,420751185,江苏 徐州,52.0,2018_10,567119376040
1,389335512,浙江 嘉兴,11676.0,2018_10,568286063319
2,1954193971,日本,3976.0,2018_10,45051617908
3,3697925558,浙江 杭州,1386.0,2018_10,565051815898
4,2568052038,香港 新界,4897.0,2018_10,566784617964


In [44]:
import re
def funa(x):
    if ' ' in x['位置']:
        s = re.split(' ',x['位置'])
        return s[0]
    elif 'None' in x['位置']:
        return x['位置']
    elif '澳大' in x['位置']:
        return x['位置']
    elif '澳门' in x['位置']:
        return x['位置'][:2]
    elif len(x['位置']) ==4:
        return x['位置'][0:2]
    elif '香港' in x['位置']:
        return x['位置'][:2]
    elif '黑龙江' in x['位置']:
        return x['位置'][:3]
    elif '河北'  in x['位置']:
        return x['位置'][:2]
    else:
        return x['位置']
    
    
    
    
def func(x):
    if ' ' in x['位置']:
        s = re.split(' ',x['位置'])
        return s[1]
    elif 'None' in x['位置']:
        return x['位置']
    elif '澳大' in x['位置']:
        return x['位置']
    elif '澳门' in x['位置']:
        return x['位置'][-4:len(x['位置'])]
    elif len(x['位置']) ==4:
        return x['位置'][2:]
    elif '香港' in x['位置']:
        return x['位置'][2:]
    elif '黑龙江' in x['位置']:
        return x['位置'][3:]
    elif '河北'  in x['位置']:
        return x['位置'][2:]
    else:
        return x['位置']

tmdf1['省份'] = tmdf1.apply(funa,axis=1)
tmdf1['城市'] = tmdf1.apply(func,axis=1)

E:\software\ANACONDA\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\software\ANACONDA\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [61]:
tmdf1.pivot_table(index=['省份','城市'],values=['店铺id','销售额'],aggfunc={'店铺id':lambda x:len(x.unique()),'销售额':np.sum},margins=1,fill_value=0)

店铺id           销售额
省份   城市                      
None None    43  5.156570e+04
上海   上海     366  9.454835e+08
丹麦   丹麦       1  2.090000e+02
云南   昆明      13  4.232633e+07
加拿大  加拿大      1  1.726600e+02
北京   北京      69  2.724493e+07
台湾   台北      13  1.441529e+06
     新北       2  3.720960e+05
     新竹       1  1.557000e+04
     桃园       3  6.414300e+04
吉林   长春       1  1.105975e+04
四川   德阳       1  2.076000e+04
     成都      20  3.836654e+06
     泸州       1  9.216560e+03
     绵阳       2  1.089920e+05
天津   天津      12  2.231395e+06
奥兰   群岛       1  1.157400e+04
安徽   亳州       1  5.900000e+01
     六安       3  1.615000e+04
     合肥      64  3.091327e+07
     芜湖      13  7.327061e+06
     蚌埠       4  2.018260e+05
     马鞍山      1  5.900000e+01
     黄山       1  3.450000e+02
山东   临沂       1  6.900000e+02
     德州       1  3.177000e+02
     日照       2  5.796810e+04
     济南      27  1.629622e+07
     淄博       2  5.912000e+03
     滨州       1  1.154300e+04
...         ...           ...
浙江   金华      22  1.424706e+08
海外   海外      16  1.221494e+07
湖北   武汉      28  1.183066e+07
     荆门       1  5.499000e+03
湖南   岳阳       4  4.380639e+04
     湘潭       1  1.821700e+04
     邵阳       1  6.612000e+03
     长沙      35  3.417083e+07
澳大利亚 澳大利亚    12  3.133161e+06
澳门   澳门半岛     3  1.286700e+05
甘肃   兰州       1  1.836180e+05
福建   厦门       9  1.177762e+06
     泉州       1  2.160000e+03
     漳州       5  1.822309e+06
     福州      24  1.816577e+07
美国   美国      15  5.901454e+05
英国   英国       3  2.201200e+04
荷兰   荷兰       1  7.601446e+04
西班牙  西班牙      7  2.323972e+06
贵州   贵阳       1  1.453382e+05
辽宁   丹东       1  4.291200e+03
     沈阳       8  3.015579e+05
重庆   重庆       9  2.939601e+05
陕西   西安       8  1.105365e+06
韩国   韩国      43  1.143828e+08
香港   九龙      27  3.867860e+07
     新界      56  1.278962e+08
     香港岛     69  4.196827e+07
黑龙江  哈尔滨      1  4.366600e+03
All        1694  3.404017e+09

[115 rows x 2 columns]

#### 淘宝操作

In [83]:
tbdf1 = tbdf[['店铺id','位置','销售额','日期','商品id']]
tbdf1[tbdf1.位置 == '日本'].销售额.sum()

44422016.82

In [84]:
# 淘宝操作
import re
import numpy as nm
shlist = ['河北','山西','辽宁','吉林','黑龙江','江苏','浙江','安徽','福建',
          '江西','山东','河南','湖北','湖南','广东','海南','四川','贵州','云南','陕西','甘肃','青海','台湾','内蒙古','广西','西藏',
          '宁夏','新疆'
]
def funb(x):
    if ' ' in x['位置']:
        s = re.split(' ',x['位置'])
        return s[0]
    elif 'None' in x['位置']:
        return x['位置']
    elif '澳大' in x['位置']:
        return x['位置']
    elif '澳门' in x['位置']:
        return x['位置'][:2]
    elif len(x['位置']) ==4:
        return x['位置'][0:2]
    elif '香港' in x['位置']:
        return x['位置'][:2]
    elif '黑龙江' in x['位置']:
        return x['位置'][:3]
    elif '河北'  in x['位置']:
        return x['位置'][:2]
    elif '内蒙古'  in x['位置']:
        return x['位置'][:3]
    elif '云南'  in x['位置']:
        return x['位置'][:2]
    elif '吉林'  in x['位置']:
        return x['位置'][:2]
    elif '四川'  in x['位置']:
        return x['位置'][:2]
    elif '安徽'  in x['位置']:
        return x['位置'][:2]
    else:
        for i in shlist:
            if i in x['位置']:
                return x['位置'][:len(i)]
        return x['位置']
        
    
    
    
    
def fund(x):
    if ' ' in x['位置']:
        s = re.split(' ',x['位置'])
        return s[1]
    elif 'None' in x['位置']:
        return x['位置']
    elif '澳大' in x['位置']:
        return x['位置']
    elif '澳门' in x['位置']:
        return x['位置'][-4:len(x['位置'])]
    elif len(x['位置']) ==4:
        return x['位置'][2:]
    elif '香港' in x['位置']:
        return x['位置'][2:]
    elif '黑龙江' in x['位置']:
        return x['位置'][3:]
    elif '河北'  in x['位置']:
        return x['位置'][2:]
    elif '内蒙古'  in x['位置']:
        return x['位置'][3:]
    elif '云南'  in x['位置']:
        return x['位置'][2:]
    elif '吉林'  in x['位置']:
        return x['位置'][2:]
    elif '四川'  in x['位置']:
        return x['位置'][2:]
    elif '安徽'  in x['位置']:
        return x['位置'][2:]
    else:
        for i in shlist:
            if i in x['位置']:
                return x['位置'][len(i):]
        return x['位置']

tbdf1['省份'] = tbdf1.apply(funb,axis=1)
tbdf1['城市'] = tbdf1.apply(fund,axis=1)


E:\software\ANACONDA\lib\site-packages\ipykernel_launcher.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\software\ANACONDA\lib\site-packages\ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [85]:
tbdf1.销售额.sum()

2203345268.36

In [88]:
import numpy as np
tbdf1.pivot_table(index=['省份','城市'],values=['店铺id','销售额'],aggfunc={'店铺id':lambda x:len(x.unique()),'销售额':np.sum},margins=1,fill_value=0)

店铺id           销售额
省份       城市                           
None     None        449  1.670605e+05
上海       上海         6283  4.249783e+08
丹麦       丹麦            2  4.291650e+03
乌克兰      乌克兰           5  4.443800e+04
云南       临沧            4  2.385920e+04
         丽江            3  4.936000e+02
         保山            3  9.727300e+02
         大理           10  4.459100e+04
         德宏           17  9.943760e+04
         文山            1  1.157200e+04
         昆明          260  4.450824e+06
         普洱            4  4.566000e+03
         曲靖            5  2.248000e+03
         楚雄            6  6.064000e+03
         玉溪            5  1.898520e+04
         红河           10  1.168600e+04
         西双版纳         32  2.278041e+06
俄罗斯      俄罗斯          17  2.018000e+04
内蒙古      乌兰察布          3  3.669000e+03
         乌海            2  2.326000e+03
         兴安            4  1.128450e+03
         包头            9  8.397000e+03
         呼伦贝尔          7  6.168100e+04
         呼和浩特         28  1.321574e+05
         巴彦淖尔          1  2.816000e+03
         赤峰            7  8.990208e+04
         通辽           13  1.364720e+05
         鄂尔多斯          2  2.678200e+04
         锡林郭勒          2  3.877000e+03
冰岛       冰岛            1  3.560000e+03
...                  ...           ...
阿拉伯联合酋长国 阿拉伯联合酋长国      1  7.600000e+02
陕西       咸阳           20  5.685333e+04
         商洛            3  1.448920e+05
         安康            1  7.480000e+02
         宝鸡           17  2.643876e+05
         延安            1  1.005000e+03
         榆林            6  3.035200e+03
         汉中           18  9.086720e+04
         渭南            3  1.999000e+03
         西安          517  6.294656e+06
         铜川            9  2.248310e+04
青海       西宁            2  2.227600e+03
韩国       韩国          441  8.416303e+06
香港       九龙          172  4.219561e+06
         新界          331  1.176520e+07
         香港岛         195  1.826097e+07
黑龙江      七台河           5  6.644000e+03
         伊春            6  3.015963e+04
         佳木斯          26  1.352611e+05
         双鸭山           2  7.262000e+02
         哈尔滨         361  7.478045e+06
         大兴安岭          2  4.380000e+02
         大庆           42  4.242897e+05
         牡丹江          49  4.405845e+05
         绥化            9  3.084400e+04
         鸡西            7  3.763589e+04
         鹤岗            3  1.189500e+04
         黑河          156  2.839703e+06
         齐齐哈尔         32  4.014357e+05
All                55374  2.203345e+09

[381 rows x 2 columns]

### （4）天猫旗舰店占比，专卖店占比，综合店占比及销量占比

In [154]:
import collections
tmmz = tmdf.店铺名称.drop_duplicates()
d = collections.Counter(tmmz.str[-3:])
d

Counter({'专营店': 373,
         '旗舰店': 791,
         '方直营': 2,
         '专卖店': 478,
         '品专营': 25,
         '外旗舰': 5,
         '猫超市': 1,
         '大药房': 1,
         '堂旗舰': 1,
         '达专卖': 2,
         '露专卖': 1,
         '肤明邸': 1,
         '妆品店': 1,
         '日照店': 1,
         '官方店': 1,
         '专属店': 1,
         '美丝琳': 1,
         '内现货': 1,
         '缘专卖': 1,
         '天世纪': 1,
         '阁专卖': 1,
         '品旗舰': 1,
         '语旗舰': 1,
         '仓直购': 1,
         '宝旗舰': 1,
         '康直营': 1,
         'ns店': 1})

In [172]:
import collections
tmmz = tmdf[['店铺名称','销售额','销量']]
tmmz['店铺标题'] = tmmz['店铺名称'].str[-3:]
tmmz.head()

E:\software\ANACONDA\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,店铺名称,销售额,销量,店铺标题
0,美一佳化妆品专营店,52.0,1,专营店
1,露得清官方旗舰店,11676.0,84,旗舰店
2,圣奈海外专营店,3976.0,71,专营店
3,innisfree海外旗舰店,1386.0,14,旗舰店
4,阿部药妆海外旗舰店,4897.0,83,旗舰店


In [182]:
import numpy as np
tmmz.pivot_table(index=['店铺标题'],values=['销售额','店铺名称'],aggfunc={'销售额':np.sum,'店铺名称':lambda x:len(x.unique())},margins=1).to_excel('E:\文档\桌面\数据分析作品-防晒\二、店铺分析\店铺名字分类.xlsx')
tmmz.pivot_table(index=['店铺标题'],values=['销售额','店铺名称'],aggfunc={'销售额':np.sum,'店铺名称':lambda x:len(x.unique())},margins=1)

,店铺名称,销售额
店铺标题,,
ns店,1,7.507800e+04
专卖店,478,3.009381e+08
专属店,1,6.300000e+04
专营店,373,3.344556e+08
仓直购,1,1.029320e+04
内现货,1,5.492341e+05
品专营,25,8.374469e+06
品旗舰,1,1.067832e+04
堂旗舰,1,2.089500e+03


### （5）月度店铺数量变化

In [5]:
import numpy as np
tmdp = tmdf.pivot_table(index=['日期'],values=['店铺id'],aggfunc=lambda x: len(x.unique()),margins=True,) #计算每月的不重复店铺数量变化
tmdp.columns=['天猫店铺数量']
tbdp = tbdf.pivot_table(index=['日期'],values=['店铺id'],aggfunc=lambda x: len(x.unique()),margins=True,) #计算每月的不重复店铺数量变化
tbdp.columns=['淘宝店铺数量']
tmdp

,天猫店铺数量
日期,
2018_1,976
2018_10,1226
2018_11,1209
2018_12,1117
2018_2,958
2018_3,1135
2018_4,1217
2018_5,1285
2018_6,1306


In [ ]:
import matplotlib.pyplot as plt
#可视化显示在页面
%matplotlib inline
#更改设计风格
plt.style.use('ggplot')

In [7]:
tmdp.plot()

In [34]:
pd.concat([tmdp,tbdp],axis=1).to_excel('E:\文档\桌面\数据分析作品-防晒\二、店铺分析\每月店铺数量变化.xlsx')
pd.concat([tmdp,tbdp],axis=1)

,天猫店铺数量,淘宝店铺数量
日期,,
2018_1,976,16610
2018_10,1226,15064
2018_11,1209,14828
2018_12,1117,12916
2018_2,958,14374
2018_3,1135,19497
2018_4,1217,22918
2018_5,1285,26293
2018_6,1306,26107
